# Inventory Risk Engine - Demo Notebook

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Gregory-307/hedge-engine/blob/main/notebooks/hedge_engine_colab.ipynb)

Real-time **inventory risk assessment** for market makers.

**What it does**: Takes your position data + market conditions, returns actionable recommendations with P&L scenarios and hedge costs.

## Setup

Install the package:

In [ ]:
# Install from GitHub (uncomment in Colab)
# !pip install -q git+https://github.com/Gregory-307/hedge-engine.git

# Local imports
import sys
sys.path.insert(0, '..')

In [ ]:
from hedge_engine.assessor import assess_inventory_risk
from hedge_engine.models import InventoryPosition, MarketConditions, RiskConfig, Action

## Basic Usage

Assess a simple long BTC position:

In [ ]:
# Define a position
position = InventoryPosition(
    asset="BTC",
    size=10.0,           # 10 BTC long
    entry_price=45000.0,
    age_minutes=60,      # 1 hour old
)

# Define market conditions
market = MarketConditions(
    current_price=45000.0,
    volatility_1d=0.03,       # 3% daily vol (normal)
    spot_spread_bps=2.0,      # 2bps spread
    perp_funding_rate=0.05,   # 5% annual funding
    bid_depth_usd=5_000_000,  # $5M liquidity
    ask_depth_usd=5_000_000,
)

# Get recommendation
result = assess_inventory_risk(position, market)

print(f"Action: {result.action.value}")
print(f"Risk Score: {result.risk_score:.1f}/100")
print(f"Hedge %: {result.hedge_pct * 100:.0f}%")
print(f"\nReasoning: {result.reasoning}")

## P&L Scenarios

Every assessment includes P&L projections for different market moves:

In [ ]:
pnl = result.pnl_scenarios

print("P&L Scenarios (10 BTC @ $45,000):")
print(f"  Price drops 5%: ${pnl.move_down_5pct:,.0f}")
print(f"  Price drops 2%: ${pnl.move_down_2pct:,.0f}")
print(f"  Price rises 2%: ${pnl.move_up_2pct:,.0f}")
print(f"  Price rises 5%: ${pnl.move_up_5pct:,.0f}")

## High Risk Scenario

Let's simulate a stressful situation: high volatility, aging position, price moved against us:

In [ ]:
# Stressed position
stressed_position = InventoryPosition(
    asset="BTC",
    size=20.0,            # Larger position
    entry_price=48000.0,  # Entered higher
    age_minutes=400,      # 6+ hours old
)

# Stressed market
stressed_market = MarketConditions(
    current_price=44000.0,    # Price dropped
    volatility_1d=0.09,       # 9% vol (extreme)
    spot_spread_bps=15.0,     # Wide spreads
    perp_funding_rate=0.15,   # High funding
    bid_depth_usd=2_000_000,  # Low liquidity
    ask_depth_usd=2_000_000,
)

stressed_result = assess_inventory_risk(stressed_position, stressed_market)

print(f"Action: {stressed_result.action.value}")
print(f"Risk Score: {stressed_result.risk_score:.1f}/100")
print(f"Hedge %: {stressed_result.hedge_pct * 100:.0f}%")
print(f"\nReasoning: {stressed_result.reasoning}")

if stressed_result.suggested_hedge:
    hedge = stressed_result.suggested_hedge
    print(f"\nSuggested Hedge:")
    print(f"  Instrument: {hedge.instrument.value}")
    print(f"  Size: {hedge.size:.2f} BTC")
    print(f"  Cost: ${hedge.total_cost_usd:.2f} ({hedge.total_cost_bps:.1f}bps)")

## Short Position Example

The engine handles short positions correctly - they profit when price drops:

In [ ]:
# Short ETH position in a squeeze
short_position = InventoryPosition(
    asset="ETH",
    size=-50.0,           # 50 ETH SHORT
    entry_price=2400.0,
    age_minutes=180,
)

squeeze_market = MarketConditions(
    current_price=2600.0,     # Price went UP (bad for shorts)
    volatility_1d=0.06,
    spot_spread_bps=5.0,
    perp_funding_rate=-0.05,  # Negative funding (shorts pay)
    bid_depth_usd=3_000_000,
    ask_depth_usd=3_000_000,
)

short_result = assess_inventory_risk(short_position, squeeze_market)

print(f"Short ETH Position Assessment:")
print(f"  Action: {short_result.action.value}")
print(f"  Risk Score: {short_result.risk_score:.1f}/100")
print(f"\nP&L if price drops 5%: ${short_result.pnl_scenarios.move_down_5pct:,.0f} (profit)")
print(f"P&L if price rises 5%: ${short_result.pnl_scenarios.move_up_5pct:,.0f} (loss)")

## Custom Risk Configuration

Override default risk parameters for different strategies:

In [ ]:
# Conservative config: lower loss tolerance
conservative_config = RiskConfig(
    max_loss_pct=0.02,        # Only 2% max loss (vs 5% default)
    hedge_trigger_loss_pct=0.01,
    max_hold_minutes=120,     # 2 hour max hold
)

# Same position, different risk appetite
conservative_result = assess_inventory_risk(position, market, conservative_config)
default_result = assess_inventory_risk(position, market)

print("Same position, different risk configs:")
print(f"\nDefault config:")
print(f"  Risk Score: {default_result.risk_score:.1f}")
print(f"  Action: {default_result.action.value}")

print(f"\nConservative config:")
print(f"  Risk Score: {conservative_result.risk_score:.1f}")
print(f"  Action: {conservative_result.action.value}")

## Risk Score Breakdown

The risk score (0-100) is composed of four factors:

| Factor | Max Points | Description |
|--------|------------|-------------|
| Loss Severity | 35 | Potential loss vs threshold, adjusted for volatility |
| Position Age | 20 | Time held vs max hold limit |
| Volatility | 25 | Current market vol vs extreme threshold |
| Liquidity | 20 | Position size vs available depth |

In [ ]:
# Visualize action thresholds
import matplotlib.pyplot as plt

thresholds = [
    (0, 40, 'HOLD', 'green'),
    (40, 55, 'REDUCE', 'yellowgreen'),
    (55, 70, 'HEDGE_PARTIAL', 'yellow'),
    (70, 85, 'HEDGE_FULL', 'orange'),
    (85, 100, 'LIQUIDATE', 'red'),
]

fig, ax = plt.subplots(figsize=(12, 2))

for start, end, label, color in thresholds:
    ax.barh(0, end - start, left=start, height=0.5, color=color, edgecolor='black')
    ax.text((start + end) / 2, 0, label, ha='center', va='center', fontsize=10, fontweight='bold')

ax.set_xlim(0, 100)
ax.set_ylim(-0.5, 0.5)
ax.set_xlabel('Risk Score', fontsize=12)
ax.set_yticks([])
ax.set_title('Action Thresholds by Risk Score', fontsize=14)

# Mark our examples
ax.axvline(x=default_result.risk_score, color='blue', linestyle='--', linewidth=2, label=f'Normal ({default_result.risk_score:.0f})')
ax.axvline(x=stressed_result.risk_score, color='purple', linestyle='--', linewidth=2, label=f'Stressed ({stressed_result.risk_score:.0f})')
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

## Using the REST API

If running the full service:

In [ ]:
# Uncomment to test against running service
# import httpx
# 
# API_URL = "http://localhost:8000"
# 
# # Health check
# resp = httpx.get(f"{API_URL}/healthz")
# print(f"Health: {resp.json()}")
# 
# # Assess a position
# resp = httpx.post(f"{API_URL}/assess", json={
#     "position": {
#         "asset": "BTC",
#         "size": 10.0,
#         "entry_price": 45000.0,
#         "age_minutes": 60
#     },
#     "market": {
#         "current_price": 45000.0,
#         "volatility_1d": 0.03,
#         "spot_spread_bps": 2.0,
#         "perp_funding_rate": 0.05,
#         "bid_depth_usd": 5000000,
#         "ask_depth_usd": 5000000
#     }
# })
# print(f"Assessment: {resp.json()}")

## Summary

The **Inventory Risk Engine** provides:

1. **Actionable recommendations**: HOLD, REDUCE, HEDGE_PARTIAL, HEDGE_FULL, or LIQUIDATE
2. **P&L scenarios**: See your exposure at +/-2% and +/-5% moves
3. **Hedge cost estimation**: Compare spot vs perp, with spread and funding costs
4. **Risk scoring**: 0-100 score based on loss severity, age, volatility, and liquidity
5. **Configurable thresholds**: Adjust risk parameters for your strategy

### Next Steps

- Run the API: `uvicorn hedge_engine.main:app --reload`
- Try different scenarios in `examples/`
- Customize `RiskConfig` for your risk appetite